# Depths and Snow Pit Data Package Contents

(12 minutes)

Learning Objectives:
- Tools to access data.
- Code snippets to extract and prep data.

In [ ]:
# standard imports
import os
from pathlib import Path
import glob
import pandas as pd
import numpy as np

#plotting imports
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use(['seaborn-notebook'])

## Download snow depth data from NSIDC
From the SnowEx20 [Depth Probe Landing Page](https://nsidc.org/data/SNEX20_SD/versions/1), you can download data and access the [User's Guide](https://nsidc.org/sites/nsidc.org/files/SNEX20_SD-V001-UserGuide_1.pdf)
<img src="images/SD-landingPage.png">

The Community Snow Depth Probe data package is a single CSV with over 36,000 geolocated snow depths! Three different instrument types were used to measure depths and are recorded in the Measurement Tool column.

os.chmod('/home/jovyan/.netrc', 0o600) #only necessary on snowex hackweek jupyterhub## Method 1: Programmatically download snow depth data from NSIDC

In [ ]:
os.chmod('/home/jovyan/.netrc', 0o600) #only necessary on snowex hackweek jupyterhub

In [ ]:
%run './scripts/nsidc-download_SNEX20_SD.001.py' 
print('Grand Mesa 2020 Snow Depth data download complete') 

In [ ]:
# show filename:
path = Path('./data/depths/')

for filename in path.glob('*.csv'):
    print(filename.name)

### Read the Depth File

In [ ]:
df = pd.read_csv('./data/depths/SnowEx2020_SnowDepths_COGM_alldepths_v01.csv', sep=',', header=0, parse_dates=[[2,3]]) #parse the date[2] and time[3] columns such that they are read in as datetime dtypes
    
print('file has been read, and is ready to use.')

In [ ]:
# check data types for each column
df.dtypes

### Prep for Data Analysis

In [ ]:
# rename some columns for ease further down
df.rename(columns = {
    'Measurement Tool (MP = Magnaprobe; M2 = Mesa 2; PR = Pit Ruler)':'Measurement Tool', 
    'Date (yyyymmdd)_Time (hh:mm, local, MST)': "Datetime"},
          inplace = True)

# set up filter for IOP date range
start = pd.to_datetime('1/28/2020') #first day of GM IOP campaign
end   = pd.to_datetime('2/12/2020') #last day of GM IOP campaign

# filter the IOP date range
df = df[(df['Datetime'] >= start) & (df['Datetime'] <= end)]

print('DataFrame shape is: ', df.shape)
df.head()

#### Use .groupby() to sort the data set

In [ ]:
# group data by the measurement tool 
gb = df.groupby('Measurement Tool', as_index=False).mean().round(1)

# show mean snow depth from each tool
gb[['Measurement Tool', 'Depth (cm)']]

#### ***Your turn***

In [ ]:
# group data by the snow pit ID


# show mean snow depth around each snow pit


#(hint: what is the pit id column called? If you're not sure you can use df.columns to see a list of column names. Consider using .head() to display only the first 5 returns)

#### Find depths associated with a certain measurement tool

In [ ]:
print('List of Measurement Tools: ', df['Measurement Tool'].unique())

In [ ]:
r = df.loc[df['Measurement Tool'] == 'PR']
print('DataFrame shape is: ', r.shape)
r.head()

#### ***Your turn***

In [ ]:
# find all depths recorded by the Mesa2


# (hint: copy/paste is your friend)

Let's make sure we all have the same pd.DataFrame() again

In [ ]:
# pit ruler snow depths from Grand Mesa IOP
r = df.loc[df['Measurement Tool'] == 'PR'] 
print( 'DataFrame is back to only pit ruler depths')

### Plotting

In [ ]:
# plot pit ruler depths 
ax = r.plot(x='Easting', y='Northing', c='Depth (cm)', kind='scatter', alpha=0.7, colorbar=True, colormap='PuBu', legend=True)
ax.set_title('Grand Mesa Pit Ruler Depths')
ax.set_xlabel('Easting [m]')
ax.set_ylabel('Northing [m]')
plt.show()

print('Notice the point on the far right - that is the "GML" or Grand Mesa Lodge pit where all instruments were deployed for a comparison study. pitID=GML')

In [ ]:
# plot histogram of pit ruler depths
ax = r['Depth (cm)'].plot.hist(bins=25)
ax.grid()
ax.set_title('Grand Mesa Pit Ruler Depths')
ax.set_xlabel('Snow Depth (cm)')
ax.set_ylabel('Frequency')

## Download snow pit data from NSIDC
From the SnowEx20 [Snow Pit Landing Page](https://nsidc.org/data/SNEX20_GM_SP/versions/1), you can download data and access the [User's Guide](https://nsidc.org/data/SNEX20_GM_SP/versions/1). 

<img src="images/SP-landingPage.png">

## Method 1: Programmatically download snow pit data from NSIDC

In [ ]:
# load snow pit data
#%run 'scripts/nsidc-download_SNEX20_GM_SP.001.py'
#print('Grand Mesa 2020 Snow Pit data download complete')

## Method 2: Access tutorial data from zenodo

We've archived datasets used for 2021 Hackweek tutorials on [Zenodo](https://zenodo.org), to ensure that these tutorials can be run in the future. The following code pulls data from the Zenodo 'record' and unzips it:

In [ ]:
%%bash 

cd /tmp
#wget -nc https://zenodo.org/record/RECORD_ID/files/snowex-hackweek/tutorial-data.zip -O tutorial-data.zip
wget -nc https://zenodo.org/record/3600624/files/dshean/hma_mb_paper-v1.0.zip -O tutorial-data.zip
unzip -n tutorial-data.zip

# temporary equivalent
#aws s3 sync s3://snowex-data/zenodo-data/ /tmp/tutorial-data 

### Don't want to work with all the files? Method to filter files

In [ ]:
# what files would you like to find?
parameter = 'temperature'
pitID = '5N19'
date = '20200128'

path = '/tmp/tutorial-data/core-datasets/pits/SnowEx20_SnowPits_GMIOP_{}_{}_{}_v01.csv'.format(date, pitID, parameter)

### Read the Pit Parameter File

In [ ]:
t = pd.read_csv(path, header=7)
t

### Plotting

In [ ]:
# plot temperature
ax = t.plot(x='Temperature (deg C)',y='# Height (cm)', legend=False)
ax.set_aspect(0.4)
ax.grid()
ax.set_title('pit {}: Temperature Profile'.format(pitID))
ax.set_xlabel('Temperature (deg C)')
ax.set_ylabel('Snow Depth (cm)')

In [ ]:
# grab a different pit parameter file
parameter = 'density'
path = '/tmp/tutorial-data/core-datasets/pits/SnowEx20_SnowPits_GMIOP_{}_{}_{}_v01.csv'.format(date, pitID, parameter)
d = pd.read_csv(path, header=7)
d

In [ ]:
# get the average density 
d['Avg Density (kg/m3)'] = d[['Density A (kg/m3)', 'Density B (kg/m3)', 'Density C (kg/m3)']].mean(axis=1, skipna=True)
d

In [ ]:
def plot_density(ax, dataframe):
    
    '''
    This function helps you plot density profiles from snow pits. Use it to iterate through 
    DataFrame rows and plot the density for each top and bottom segment.
    
    '''
    
    for index, row in dataframe.iterrows():
        # plot blue bars to represent 10cm density intervals
        top = row['# Top (cm)']
        bottom = row['Bottom (cm)']
        dens = row["Avg Density (kg/m3)"]
        ax.plot([dens, dens],[bottom, top], color='blue', linewidth=4)

        # plot a red cross to show the spread between A and B samples
        densA = row["Density A (kg/m3)"]
        densB = row["Density B (kg/m3)"]
        middle = bottom + 5.
        ax.plot([densA, densB],[middle,middle], color='red')
        
    return ax

fig, ax = plt.subplots()
ax = plot_density(ax, d)
ax.set_xlim(50, 400)
ax.set_ylim(0, 140)
ax.grid(color='lightgray', alpha=.5)
ax.set_aspect(2)
ax.set_title('pit {}: Density Profile'.format(pitID))
ax.set_xlabel('Density kg/m3')
ax.set_ylabel('Snow Depth (cm)')   